# Neo4j와 LangChain을 활용한 영화 추천 시스템


---

## 1. Neo4J DB 환경 설정

In [1]:
import os
from dotenv import load_dotenv

# 환경 변수 로드
load_dotenv(override=True)

True

In [2]:
from langchain_neo4j import Neo4jGraph

# LangChain 도구 활용 - DB 연결 객체 초기화 
graph = Neo4jGraph( 
    url=os.getenv("NEO4J_URI"), 
    username=os.getenv("NEO4J_USERNAME"), 
    password=os.getenv("NEO4J_PASSWORD"),
    database=os.getenv("NEO4J_DATABASE")
)

In [3]:
# 테스트 쿼리 실행 
cypher_query = """
MATCH (n:Movie)
RETURN COUNT(n) AS Movie_Count
"""

graph.query(cypher_query)

[{'Movie_Count': 4803}]

---

## 2. 전문 검색(Full-Text Search) 활용

Neo4j의 전문 검색 기능은 그래프 데이터베이스의 관계 기반 쿼리 기능과 결합되어, 단순한 키워드 검색을 넘어 맥락과 관계를 고려한 지능적인 검색 솔루션을 구현할 수 있게 합니다. 이는 영화 추천 시스템, 지식 그래프 검색, 콘텐츠 관리 시스템 등 다양한 응용 분야에서 활용될 수 있습니다.

1. **텍스트 분석과 토큰화**:
   - 전문 검색 인덱스는 텍스트를 단어(토큰)로 분리하고 분석합니다.
   - 대소문자 구분 없이 검색이 가능합니다.

2. **검색 문법 지원**:
   - 와일드카드: `star*`는 "star", "starship", "stargate" 등을 검색할 수 있습니다.
   - 부정 검색: `-war`는 "war"가 포함되지 않은 결과를 반환합니다.
   - 필수 검색어: `+lord +rings`는 두 단어가 모두 포함된 결과만 반환합니다.
   - 구문 검색: `"lord of the rings"`는 정확한 구문을 포함하는 결과를 반환합니다.

3. **그래프 구조와의 통합**:
   - 텍스트 검색 결과를 첫 단계로 사용하고, 이후 그래프 탐색을 통해 관련 데이터를 추출할 수 있습니다.
   - 이는 관계형 데이터베이스에서는 구현하기 어려운 강력한 기능입니다.

4. **관련성 점수(Relevance Score)**:
   - 검색 결과는 관련성 점수와 함께 제공되어 가장 적합한 결과를 우선 표시할 수 있습니다.
   - 이 점수는 검색어와 문서 내용의 일치도, 단어 빈도 등을 고려하여 계산됩니다.



### 2.1 전문 검색 인덱스 설정

- **`Movie` 노드의 `title` 속성**에 대한 전문 검색 인덱스를 생성

   - **CREATE FULLTEXT INDEX**: 전문 검색 인덱스 생성 명령어입니다.
   - **movie_title_fulltext**: 생성할 인덱스의 이름입니다.
   - **IF NOT EXISTS**: 이미 동일한 이름의 인덱스가 있으면 생성하지 않습니다.
   - **FOR (m:Movie)**: `Movie` 라벨을 가진 노드를 대상으로 합니다.
   - **ON EACH [m.title]**: `title` 속성을 인덱싱합니다. 

In [5]:
# Movie 노드의 title 속성에 대한 전문 검색 인덱스 생성
# CREATE FULLTEXT INDEX: 전문 검색(Full-Text Search)을 위한 인덱스를 생성하는 Cypher 명령어
# movie_title_fulltext: 생성할 인덱스의 이름 (나중에 이 이름으로 인덱스를 참조)
# IF NOT EXISTS: 동일한 이름의 인덱스가 이미 존재하는 경우 오류 없이 건너뜀
# FOR (m:Movie): Movie 라벨을 가진 모든 노드에 대해 인덱스 적용
# ON EACH [m.title]: 각 Movie 노드의 title 속성만 인덱싱 (여러 속성을 인덱싱하려면 배열에 추가 가능)
fulltext_index_query = """
CREATE FULLTEXT INDEX movie_title_fulltext IF NOT EXISTS 
FOR (m:Movie) ON EACH [m.title]
"""
graph.query(fulltext_index_query)

[]

In [6]:
graph.query("SHOW FULLTEXT INDEXES") # 생성된 인덱스 확인

[{'id': 8,
  'name': 'movie_title_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

- **`Movie` 노드의 `title`, `tagline` 속성**에 대한 전문 검색 인덱스를 생성

   - **ON EACH [m.title, m.tagline]**: 여러 속성을 포함하려면 `[m.title, m.tagline]`와 같이 배열에 추가하면 함께 인덱싱 처리됨

In [7]:
# Movie 노드의 title과 tagline 속성 모두에 대한 전문 검색 인덱스 생성
# CREATE FULLTEXT INDEX: 전문 검색(Full-Text Search)을 위한 인덱스를 생성하는 Cypher 명령어
# movie_title_tagline_fulltext: 생성할 인덱스의 이름 (두 속성을 모두 포함하는 인덱스임을 이름에 반영)
# IF NOT EXISTS: 동일한 이름의 인덱스가 이미 존재하는 경우 오류 없이 건너뜀
# FOR (m:Movie): Movie 라벨을 가진 모든 노드에 대해 인덱스 적용
# ON EACH [m.title, m.tagline]: 각 Movie 노드의 title과 tagline 두 속성을 모두 인덱싱
# 이렇게 하면 영화 제목뿐만 아니라 영화의 태그라인(짧은 홍보문구)에서도 검색 가능
fulltext_index_query = """
CREATE FULLTEXT INDEX movie_title_tagline_fulltext IF NOT EXISTS 
FOR (m:Movie) ON EACH [m.title, m.tagline]
"""
graph.query(fulltext_index_query)

[]

In [8]:
graph.query("SHOW FULLTEXT INDEXES") # 생성된 인덱스 확인

[{'id': 8,
  'name': 'movie_title_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 9,
  'name': 'movie_title_tagline_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title', 'tagline'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': None}]

- **`Person` 노드의 `name` 속성**에 대한 전문 검색 인덱스를 생성

- 검색 예시:
    1. 특정 이름을 가진 배우/감독 검색
    2. 유사한 이름의 배우/감독 검색 (퍼지 검색 사용)
    3. 배우/감독 이름의 부분 문자열 검색

In [9]:
# Person 노드의 name 속성에 전문 검색 인덱스 생성
# CREATE FULLTEXT INDEX: 전문 검색(Full-Text Search)을 위한 인덱스를 생성하는 Cypher 명령어
# person_name_fulltext: 생성할 인덱스의 이름으로, 나중에 이 이름으로 인덱스를 참조할 수 있음
# IF NOT EXISTS: 동일한 이름의 인덱스가 이미 존재하는 경우 오류 없이 건너뜀
# FOR (p:Person): Person 라벨을 가진 모든 노드에 대해 인덱스 적용
# ON EACH [p.name]: 각 Person 노드의 name 속성만 인덱싱 
person_fulltext_index_query = """
CREATE FULLTEXT INDEX person_name_fulltext IF NOT EXISTS
FOR (p:Person) ON EACH [p.name]
"""
graph.query(person_fulltext_index_query)

[]

In [10]:
graph.query("SHOW FULLTEXT INDEXES") # 생성된 인덱스 확인

[{'id': 8,
  'name': 'movie_title_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 9,
  'name': 'movie_title_tagline_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Movie'],
  'properties': ['title', 'tagline'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0},
 {'id': 10,
  'name': 'person_name_fulltext',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'FULLTEXT',
  'entityType': 'NODE',
  'labelsOrTypes': ['Person'],
  'properties': ['name'],
  'indexProvider': 'fulltext-2.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

### 2.2 검색 쿼리

- **기본 전문 검색 (`love`라는 단어가 포함된 영화 제목을 검색)**:

    - **CALL db.index.fulltext.queryNodes**: 전문 검색 인덱스를 사용하여 노드를 검색하는 내장 도구입니다.
    - **"movie_title_fulltext"**: 사용할 인덱스의 이름입니다.
    - **$search_term**: 검색어 매개변수입니다 (예: "love").
    - **YIELD node, score**: 검색 결과로 노드와 관련성 점수를 반환합니다.
    - **ORDER BY SearchRelevance DESC**: 관련성 점수가 높은 순서대로 결과를 정렬합니다.


In [11]:
# `love`라는 단어가 포함된 영화 제목을 검색
# 전문 검색 인덱스를 사용하여 영화 제목에서 'love' 키워드 검색
search_query = """
// db.index.fulltext.queryNodes: 전문 검색 인덱스를 사용하여 노드를 검색하는 프로시저
// "movie_title_fulltext": 앞서 생성한 영화 제목 전문 검색 인덱스 이름
// $search_term: 파라미터로 전달되는 검색어 (여기서는 "love")
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)

// YIELD: 프로시저의 결과를 반환
// node: 검색된 노드 객체, score: 검색 관련성 점수
YIELD node, score

// RETURN: 결과로 반환할 데이터 지정
// node.title: 검색된 영화 노드의 제목 속성
// AS MovieTitle: 결과 컬럼명 지정
// score AS SearchRelevance: 검색 관련성 점수를 SearchRelevance로 이름 지정
RETURN node.title AS MovieTitle, score AS SearchRelevance

// ORDER BY: 결과 정렬 방식 지정
// SearchRelevance DESC: 관련성 점수 기준 내림차순 정렬(가장 관련성 높은 결과가 먼저 표시)
ORDER BY SearchRelevance DESC

// LIMIT 5: 상위 5개 결과만 반환
LIMIT 5
"""
graph.query(search_query, params={"search_term": "love"})

[{'MovieTitle': 'Love Stinks', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love & Basketball', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love Happens', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love Ranch', 'SearchRelevance': 2.281766891479492},
 {'MovieTitle': 'Love Letters', 'SearchRelevance': 2.281766891479492}]

- **기본 전문 검색 (영화 제목/태그라인 검색)**:

In [12]:
# 영화 제목과 태그라인에서 키워드 검색
search_query = """
// db.index.fulltext.queryNodes: 전문 검색 인덱스를 사용하여 노드를 검색하는 프로시저
// "movie_title_tagline_fulltext": 영화 제목과 태그라인을 위한 전문 검색 인덱스 이름
CALL db.index.fulltext.queryNodes("movie_title_tagline_fulltext", $search_term)

// YIELD: 프로시저의 결과를 반환
// node: 검색된 노드 객체, score: 검색 관련성 점수
YIELD node, score

// RETURN: 결과로 반환할 데이터 지정
// node.title: 검색된 영화의 제목
// node.tagline: 검색된 영화의 태그라인
// score: 검색 관련성 점수
RETURN node.title AS MovieTitle, node.tagline AS Tagline, score AS SearchRelevance

// ORDER BY: 결과 정렬 방식 지정
// SearchRelevance DESC: 관련성 점수 기준 내림차순 정렬
ORDER BY SearchRelevance DESC

// LIMIT 5: 상위 5개 결과만 반환
LIMIT 5
"""
graph.query(search_query, params={"search_term": "dream"})


[{'MovieTitle': 'Goal!: The Dream Begins',
  'Tagline': 'Every Dream Has A Beginning',
  'SearchRelevance': 4.898340702056885},
 {'MovieTitle': 'Opal Dream',
  'Tagline': None,
  'SearchRelevance': 3.1974334716796875},
 {'MovieTitle': 'Dream House',
  'Tagline': 'Once upon a time, there were two little girls who lived in a house.',
  'SearchRelevance': 3.1974334716796875},
 {'MovieTitle': 'The Walk',
  'Tagline': 'Dream High.',
  'SearchRelevance': 3.0625619888305664},
 {'MovieTitle': 'Superstar',
  'Tagline': 'Dare to dream.',
  'SearchRelevance': 2.84822940826416}]

- **기본 전문 검색 (배우/감독 이름 검색)**:

In [13]:
# 배우/감독 이름 검색
search_query = """
// db.index.fulltext.queryNodes: 전문 검색 인덱스를 사용하여 노드를 검색하는 프로시저
// "person_name_fulltext": 앞서 생성한 인물 이름 전문 검색 인덱스 이름
CALL db.index.fulltext.queryNodes("person_name_fulltext", $search_term)

// YIELD: 프로시저의 결과를 반환
// node: 검색된 노드 객체, score: 검색 관련성 점수
YIELD node, score

// RETURN: 결과로 반환할 데이터 지정
// node.name: 검색된 인물 노드의 이름 속성
// AS PersonName: 결과 컬럼명을 PersonName으로 지정
// score AS SearchRelevance: 검색 관련성 점수를 SearchRelevance로 이름 지정
RETURN node.name AS PersonName, score AS SearchRelevance

// ORDER BY: 결과 정렬 방식 지정
// SearchRelevance DESC: 관련성 점수 기준 내림차순 정렬(가장 관련성 높은 결과가 먼저 표시)
ORDER BY SearchRelevance DESC

// LIMIT 3: 상위 3개 결과만 반환
LIMIT 3
"""
graph.query(search_query, params={"search_term": "tom"})

[{'PersonName': 'Tom Noonan', 'SearchRelevance': 2.360572099685669},
 {'PersonName': 'Tom Hulce', 'SearchRelevance': 2.360572099685669},
 {'PersonName': 'Tom Bosley', 'SearchRelevance': 2.360572099685669}]

- **퍼지 검색 (Fuzzy Search)**:

    - 검색어와 **완전히 일치하지 않아도** 유사한 단어나 표현을 인식함
    - 문자열 간 **편집 거리(edit distance)** 개념을 활용하여 유사성을 판단함
    - 사용자 경험을 향상시키고 **검색 실패율을 감소**시키는 효과가 있음
    - Neo4j에서 전문 검색 인덱스와 함께 사용하여 **더 포괄적인 검색 결과**를 제공함

In [14]:
# 퍼지 검색 - 오타를 허용하는 검색 (예: 'afair'로 'affair' 찾기)
fuzzy_search_query = """
// db.index.fulltext.queryNodes: 전문 검색 인덱스를 사용하여 노드를 검색하는 프로시저
// "movie_title_fulltext": 영화 제목에 대한 전문 검색 인덱스 이름
// $search_term: 파라미터로 전달된 검색어 (여기서는 "afair~0.7"로 전달됨)
// ~0.7: 퍼지 검색 표시자로, 검색어와 70% 이상 일치하는 결과를 반환
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)

// YIELD: 프로시저의 결과를 반환
// node: 검색된 노드 객체, score: 검색 관련성 점수
YIELD node, score

// RETURN: 결과로 반환할 데이터 지정
// node.title: 검색된 영화 노드의 제목 속성
// AS MovieTitle: 결과 컬럼명을 MovieTitle로 지정
// score AS SearchRelevance: 검색 관련성 점수를 SearchRelevance로 이름 지정
RETURN node.title AS MovieTitle, score AS SearchRelevance

// ORDER BY: 결과 정렬 방식 지정
// SearchRelevance DESC: 관련성 점수 기준 내림차순 정렬(가장 관련성 높은 결과가 먼저 표시)
ORDER BY SearchRelevance DESC

// LIMIT 5: 상위 5개 결과만 반환
LIMIT 5
"""

# graph.query: Neo4j 데이터베이스에 쿼리를 실행하는 함수
# params={"search_term": "afair~0.7"}: 퍼지 검색을 위한 파라미터 전달
# ~0.7은 편집 거리(edit distance)를 기반으로 70% 유사도를 가진 결과까지 포함
fuzzy_results = graph.query(fuzzy_search_query, params={"search_term": "afair~0.7"})

# 검색 결과를 순회하며 영화 제목과 관련도 점수를 출력
for result in fuzzy_results:
    print(f"{result['MovieTitle']} (관련도: {result['SearchRelevance']})")

Vanity Fair (관련도: 2.685884475708008)
Fair Game (관련도: 2.685884475708008)
State Fair (관련도: 2.685884475708008)
The Thomas Crown Affair (관련도: 2.3315298557281494)
My Fair Lady (관련도: 2.3031468391418457)


- **와일드카드 검색 (Wildcard Search)**:

    - 와일드카드 검색은 특정 패턴으로 시작하거나 끝나는 단어를 검색할 때 사용
    - `"star*"` (star로 시작하는 모든 단어)

In [15]:
# 와일드카드 검색 - 특정 패턴으로 시작하는 단어 검색 (예: 'star'로 시작하는 영화)
wildcard_search_query = """
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score

RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

# star*: 'star'로 시작하는 모든 단어를 검색
wildcard_results = graph.query(wildcard_search_query, params={"search_term": "star*"})

print("\n=== 와일드카드 검색 결과 (star*) ===")
for result in wildcard_results:
    print(f"{result['MovieTitle']} ({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


=== 와일드카드 검색 결과 (star*) ===
Star Trek Beyond (2016-07-07) - 관련도: 1.0000
The Fault in Our Stars (2014-05-16) - 관련도: 1.0000
My Lucky Star (2013-09-17) - 관련도: 1.0000
20 Feet from Stardom (2013-06-14) - 관련도: 1.0000
Star Wars (1977-05-25) - 관련도: 1.0000


- **정확한 구문 검색 (Exact Phrase Search)**:

    - 단어들이 정확히 지정된 순서로 나타나는 결과만 검색
    - `"\"love story\""` (정확하게 "love story"라는 구문 검색)

In [16]:
# 정확한 구문 검색 - 정확한 문구를 검색 (예: "love story")
phrase_search_query = """
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score

RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

# \"love story\": 정확히 "love story"라는 구문을 포함하는 영화 검색
# 이스케이프 처리된 쌍따옴표를 사용하여 정확한 구문을 지정
phrase_results = graph.query(phrase_search_query, params={"search_term": "\"love story\""})

print("\n=== 정확한 구문 검색 결과 (\"love story\") ===")
for result in phrase_results:
    print(f"{result['MovieTitle']} ({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


=== 정확한 구문 검색 결과 ("love story") ===
Wristcutters: A Love Story (2006-01-24) - 관련도: 3.6628
Capitalism: A Love Story (2009-09-06) - 관련도: 3.6628


- **논리 연산자 검색 (AND/OR/NOT)**:

    - 논리 연산자를 사용하여 여러 검색어를 조합 가능
    - `"love AND story NOT horror"` (love와 story를 포함하고 horror는 제외)

In [17]:
# 논리 연산자 검색 - 특정 단어를 포함하고 다른 단어는 제외 (예: 'love'와 'story'를 포함하고 'horror'는 제외)
boolean_search_query = """
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score

RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

# love AND story NOT horror: 'love'와 'story'를 포함하고 'horror'는 제외하는 검색
boolean_results = graph.query(boolean_search_query, params={"search_term": "love AND story NOT horror"})

print("\n=== 논리 연산자 검색 결과 (love AND story NOT horror) ===")
for result in boolean_results:
    print(f"{result['MovieTitle']} ({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


=== 논리 연산자 검색 결과 (love AND story NOT horror) ===
Wristcutters: A Love Story (2006-01-24) - 관련도: 3.6628
Capitalism: A Love Story (2009-09-06) - 관련도: 3.6628


- **가중치 부여 검색 (Boosting)**:

    - 가중치 부여 검색은 특정 검색어에 더 높은 중요도를 부여하는 방법
    - `"love^4 affair"` (love에 4배 가중치 부여)

In [18]:
# 가중치 부여 검색 - 특정 단어에 더 높은 가중치 부여 (예: 'love'에 4배 가중치)
boost_search_query = """
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term)
YIELD node, score

RETURN node.title AS MovieTitle, node.released AS ReleaseDate, score AS SearchRelevance
ORDER BY SearchRelevance DESC
LIMIT 5
"""

# love^4 story: 'love'에 4배 가중치를 부여한 검색
boost_results = graph.query(boost_search_query, params={"search_term": "love^4 story"})

print("\n=== 가중치 부여 검색 결과 (love^4 story) ===")
for result in boost_results:
    print(f"{result['MovieTitle']} ({result['ReleaseDate']}) - 관련도: {result['SearchRelevance']:.4f}")


=== 가중치 부여 검색 결과 (love^4 story) ===
Love Jones (1997-03-14) - 관련도: 9.1271
Love Actually (2003-09-07) - 관련도: 9.1271
Love Ranch (2010-06-30) - 관련도: 9.1271
Endless Love (2014-02-12) - 관련도: 9.1271
Love Letters (1983-04-01) - 관련도: 9.1271


### 2.3 전문 검색과 그래프 탐색 결합

- **전문 검색 -> 그래프 탐색**:

    1. 먼저 `love`와 같은 검색어로 영화를 검색합니다.
    2. 검색된 영화에 출연한 배우들을 찾습니다 `(movie)<-[:ACTED_IN]-(actor:Person)`.
    3. 영화 제목, 검색 관련성 점수와 함께 해당 영화에 출연한 모든 배우 목록을 반환합니다.
    4. `collect(actor.name)`은 각 영화에 대한 모든 배우 이름을 배열로 수집합니다.


In [19]:
# 전문 검색 -> 그래프 탐색: 특정 단어를 포함하는 영화를 먼저 찾고(full-text), 각 영화에 출연한 출연 배우 찾기(graph)
combined_search_query = """
// db.index.fulltext.queryNodes: 전문 검색 인덱스를 사용하여 노드 검색
// "movie_title_fulltext": 영화 제목에 대한 전문 검색 인덱스 이름
// $search_term: 파라미터로 전달된 검색어
CALL db.index.fulltext.queryNodes("movie_title_fulltext", $search_term) 

// YIELD: 프로시저의 결과를 반환
// node as movie: 검색된 노드를 movie로 별칭 지정
// score: 검색 관련성 점수
YIELD node as movie, score

// MATCH: 그래프 패턴 매칭
// (movie)<-[:ACTED_IN]-(actor:Person): movie 노드에 ACTED_IN 관계로 연결된 Person 타입의 actor 노드 찾기
MATCH (movie)<-[:ACTED_IN]-(actor:Person)

// RETURN: 쿼리 결과 반환
// movie.title AS MovieTitle: 영화 제목을 MovieTitle로 별칭 지정
// score AS SearchRelevance: 검색 관련성 점수를 SearchRelevance로 별칭 지정
// collect(actor.name) AS Actors: 각 영화에 출연한 모든 배우 이름을 배열로 수집하여 Actors로 별칭 지정
RETURN movie.title AS MovieTitle, score AS SearchRelevance, 
       collect(actor.name) AS Actors

// ORDER BY SearchRelevance DESC: 검색 관련성 점수 기준 내림차순 정렬(가장 관련성 높은 결과가 먼저 표시)
ORDER BY SearchRelevance DESC

// LIMIT 5: 상위 5개 결과만 반환
LIMIT 5
"""

# graph.query: Neo4j 데이터베이스에 쿼리를 실행하는 함수
# params={"search_term": "love"}: "love"라는 검색어를 파라미터로 전달
combined_results = graph.query(combined_search_query, params={"search_term": "love"})

# 검색 결과를 순회하며 영화 제목, 관련도 점수, 출연 배우 목록을 출력
for result in combined_results:
    print(f"{result['MovieTitle']} (관련도: {result['SearchRelevance']})")
    print(f"  출연 배우: {', '.join(result['Actors'])}")
    print()

Love Stinks (관련도: 2.281766891479492)
  출연 배우: Bridgette Wilson, Bill Bellamy, Tiffani Thiessen, French Stewart, Tyra Banks

Love & Basketball (관련도: 2.281766891479492)
  출연 배우: Alfre Woodard, Omar Epps, Sanaa Lathan, Chris Warren, Jr., Kyla Pratt

Love Happens (관련도: 2.281766891479492)
  출연 배우: Martin Sheen, Jennifer Aniston, Aaron Eckhart, Judy Greer, Deirdre Blades

Love Ranch (관련도: 2.281766891479492)
  출연 배우: Joe Pesci, Bai Ling, Gina Gershon, Helen Mirren, Sergio Peris-Mencheta

Love Letters (관련도: 2.281766891479492)
  출연 배우: Jamie Lee Curtis, James Keach, Matt Clark, Bonnie Bartlett, Amy Madigan



- **두 가지 전문 인덱스를 모두 사용하는 복합 검색**:

    1. 첫 번째 서브쿼리에서 "star"가 제목에 포함된 모든 영화와 관련성 점수를 찾습니다.
    2. 두 번째 서브쿼리에서 "harrison"이 이름에 포함된 모든 인물(배우/감독)을 찾습니다.
    3. 그런 다음 메인 쿼리에서 두 서브쿼리 결과를 연결하여, Harrison이 출연한 영화 중 제목에 "star"가 포함된 영화만 필터링합니다.
    4. 결과를 영화 관련성 점수와 평점을 기준으로 정렬합니다.

In [20]:
# "star"가 제목에 포함된 영화 중 "Harrison"이 출연한 영화 검색
advanced_combined_search = """
// 첫 번째 서브쿼리: 영화 제목 전문 검색
CALL () {
    // movie_title_fulltext 인덱스를 사용하여 $movie_term(star)이 포함된 영화 노드 검색
    CALL db.index.fulltext.queryNodes("movie_title_fulltext", $movie_term)

    // YIELD: 검색 결과로 노드와 관련성 점수 반환
    // node를 movie로, score를 movieScore로 별칭 지정
    YIELD node as movie, score as movieScore

    // 검색된 영화 노드와 관련성 점수 반환
    RETURN movie, movieScore
}

// 두 번째 서브쿼리: 인물 이름 전문 검색
CALL () {
    // person_name_fulltext 인덱스를 사용하여 $actor_term(harrison)이 포함된 인물 노드 검색
    CALL db.index.fulltext.queryNodes("person_name_fulltext", $actor_term)

    // 검색된 인물 노드를 actor로 별칭 지정
    YIELD node as actor

    // 검색된 배우 노드 반환
    RETURN actor
}

// 메인 쿼리: 두 서브쿼리 결과를 연결
// 검색된 배우(actor)가 출연한(ACTED_IN) 검색된 영화(movie) 관계 매칭
MATCH (movie)<-[:ACTED_IN]-(actor)

// 최종 결과 반환
// movie.title: 영화 제목을 MovieTitle로 별칭
// movieScore: 영화 검색 관련성 점수를 MovieRelevance로 별칭
// actor.name: 배우 이름을 ActorName으로 별칭
// movie.rating: 영화 평점을 Rating으로 별칭
RETURN movie.title AS MovieTitle, movieScore AS MovieRelevance,
       actor.name AS ActorName, movie.rating AS Rating

// 결과 정렬: 영화 관련성 점수 내림차순, 평점 내림차순
ORDER BY MovieRelevance DESC, Rating DESC

// LIMIT 5: 상위 5개 결과만 반환
LIMIT 5
"""

# Neo4j 데이터베이스에 쿼리 실행
# params: "star"와 "harrison"을 각각 movie_term과 actor_term 파라미터로 전달
star_harrison_movies = graph.query(advanced_combined_search,
                                  params={"movie_term": "star", "actor_term": "harrison"})

# 검색 결과 출력: 영화 제목, 평점, 출연 배우 정보 표시
for result in star_harrison_movies:
    print(f"{result['MovieTitle']} ({result['Rating']}) - 출연 배우: {result['ActorName']}")
print()

Star Wars (8.1) - 출연 배우: Harrison Ford

